In [1]:
# Notebook을 실행한 브라우저에서 바로 그림을 볼수 있도록
%matplotlib inline
import random # used for 1) generation of synthetic data or 2) initializations of model parameters
import time # d2l에 들어있다.
import numpy as np
import tensorflow as tf
from IPython import display # d2l에 들어있다.
from d2l import tensorflow as d2l

In [2]:
# Timer() ftn for calculating the time spent for a given operation
class Timer(): #@save
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return the average time."""
        return tf.reduce_sum(self.times) / len(self.times)

    def sum(self):
        """Return the sum of time."""
        return tf.reduce_sum(self.times)

    def cumsum(self):
        """Return the accumulated time."""
        return np.array(self.times).cumsum().tolist() # 리스트 형식은 연산이 되지 않아서 한번 변환이 이루어졌습니다.

In [3]:
# User-defined function for plotting
def use_svg_display(): #@save
    """Use the svg format to display a plot in the Jupyter."""
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)): #@save
    """Set the figure size for matplotlib."""
    use_svg_display()
    d2l.plt.rcParams['figure.figsize'] = figsize

#@save
def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
    """Set the axes for matplotlib."""
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend:
        axes.legend(legend)
    axes.grid() # 그리드 선을 구성하십시오. 격자무늬를 의미하는 것 같습니다.

#@save
def plot(X, Y=None, xlabel=None, ylabel=None, legend=None, xlim=None,
ylim=None, xscale='linear', yscale='linear',
fmts=('-', 'm--', 'g-.', 'r:'), figsize=(3.5, 2.5), axes=None):
    """Plot data points."""
    if legend is None:
        legend = []
    
    set_figsize(figsize)
    axes = axes if axis else d2l.plt.gca() # 'd2l.plt.gca()'로 현재의 axes 객체를 구할 수 있습니다.

    # Return True if 'X' (tensor or list) has 1 axis.
    def has_one_axis(X):
        return(hasattr(X, "ndim") and X.ndim == 1 or isinstance(X, list) 
        and not hasattr(X[0], "__len__"))

    if has_one_axis(X):
        X = [X] # It has len(X) = 1 after an operation.
    if Y is None:
        X, Y = [[]] * len(X), X
    elif has_one_axis(Y):
        Y = [Y]
    if len(X) != len(Y):
        X = X * len(Y) # Same support임을 guarantee 해줍니다.
    axes.cla() # 현재의 좌표축을 지웁니다.
    for x, y, fmt in zip(X, Y, fmts):
        if len(x):
            axes.plot(x, y, fmt)
        else:
            axes.plot(y, fmt) # 얘는 일종의 방어적 프로그래밍으로 이해할 수 있다고 생각합니다.
    set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend)

In [5]:
# Generating the Dataset
# w : tensor of weight
# b : bias (length 1 - since linear regression setting)
# num_examples : number of (training) samples to generate
def synthetic_data(w, b, num_examples): #@save
    """Generate y = Xw + b + noise."""
    X = tf.zeros((num_examples, w.shape[0])) # 'w.shape[0]'을 하는 이유는 w가 tensor이기 때문입니다. 궁금하다면 관련 코드를 직접 작성하여 실행해보면 될 것 같습니다.
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1,1))) + b # Regression function. 왜 'tf.reshape()'을 사용하는지는 'w.shape()'를 한번 사용해보면 알 수 있습니다.
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

In [6]:
# Partitioning the Dataset into Minibatches
# Training set이 아닌 경우에는 shuffling이 불필요하다고 생각하는 것 같습니다.
# data_arrays : tuple of (features, labels) 
# batch_size : size of the batch
# is_train : if True, then shuffle.
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a Tensorflow data iterator.""" # yield문을 이용해서 generator를 반환하는 것과 유사한 / 동일한 목적입니다.
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size=batch_size)
    return dataset # Now, the time for iterator to do his/her work (e.g. next(iter(data_iter))). 

In [7]:
# Defining the Model ('net')
# There exist two versions : 
# 1) Manually made type
# 2) tf.keras.Sequential()에 layer을 addition함으로써 : linear model도 accommodate 가능함을 기억할 필요가 있습니다.
# 이 code chunk에는 1)에 대응되는 user-defined ftns만을 정리하여 제공할 계획입니다.
# 이에 따라, 2)의 경우에는 원형이 되는 prototype만을 아래와 같이 주석으로 제공하도록 하겠습니다.
# 추가적으로, 2)의 경우에는 initialization of model parameters for training도 함께 가능하도록 작성할 수 있습니다.
"""
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))
"""

def linreg(X, w, b): #@save
    # The linear regression model.
    return tf.matmul(X, w) + b

In [8]:
# Defining the Loss Function ('loss')
# There exist two versions :
# 1) Manually made type
# 2) tf.keras.losses에 속한 loss function들 중 하나를 이용하는 방법이 있습니다.
"""
loss = tf.keras.losses.MeanSquaredError()
"""

def squared_loss(y_hat, y): #@save
    # Squared loss.
    return (y_hat - tf.reshape(y, y_hat.reshape)) ** 2 / 2 # sum이 아닌 vector 형식으로 주어집니다.

In [ ]:
# Defining the Optimization Algorithm
# There exist two versions :
# 1) Manually made type
# 2) tf.keras.optimizers.OPTIMIZER(learning_rate=0.03)
"""
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)
"""

# params: tuple of weight and bias
# grads: tuple of gradients w.r.t weight and bias, respectively
# lr: learning rate
# batch_size: size of the batch
def sgd(params, grads, lr, batch_size): #@save
    # Minibatch stochastic gradient descent
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)